In [88]:
import string
import sys
import io 
import os
import nltk
import pandas as pd
import numpy as np
import os
from collections import Counter
from gensim.models import Word2Vec
from cade.cade import CADE
import treetaggerwrapper

In [89]:
path_autori_cade = "C:/Users/cafe6/Università degli Studi di Milano-Bicocca/g.carbone8@campus.unimib.it - data/Models/autori/frasi_NON_lemmatizzate/CADE/CADE_5"

In [90]:
# Funzione per comparare una parola fra embedding diversi

def compare_word_btw_embeddings(word_to_study, n_mod, text1, text2):
    #parola da studiare, numero di embedding creati per corpus, testo preso come riferimento (da cui estraggo il vettore della
    #parola target), testo da cui estraggo le parole simili
    
    #prendo il numero totale di embedding dei due testi considerati
    embeddings1 = []
    for it in range(n_mod):
        model = Word2Vec.load(text1 + '_' + str(it) + '.model') #sistemare il nome se necessario
        embeddings1.append(model)
        
    embeddings2 = []
    for it in range(n_mod):
        model = Word2Vec.load(text2 + '_' + str(it) + '.model')
        embeddings2.append(model)

    #ottengo la lista delle parole più simili alla target per ciascun modello
    words = []
    for k in range(n_mod):
        word_vector = embeddings1[k].wv[word_to_study]
        for tupla in embeddings2[k].wv.most_similar(positive=[word_vector])[:10]: #modificare a piacere il numero
            words.append(tupla[0])
    
    #creo dizionario con parole uniche e relativa frequenza assoluta
    c = Counter(words)
    word_frequencies = dict(c)
    
    #creo dizionario con parole uniche calcolando la similarità media
    init = [0] * len(np.unique(words))
    word_similarities = dict(zip(np.unique(words), init))
    
    for k in range(n_mod):
        word_vector = embeddings1[k].wv[word_to_study]
        for tupla in embeddings2[k].wv.most_similar(positive=[word_vector])[:10]:
            word_similarities[tupla[0]] += tupla[1]
    
    for word in np.unique(words):
        word_similarities[word] = word_similarities[word] / word_frequencies[word]
  
    #genero il dataframe finale
    df = pd.DataFrame(list(word_similarities.items()), columns=['Words','Med.similarity'])
    df = df.sort_values(by=['Med.similarity'], ascending=False)
    df = df.head(n=10)
    
    return df

In [91]:
def similar_words_per_topos(word, n_mod, n_words, text):
        #parola target, numero di modelli, numero di parole simili da estrarre, corpus da analizzare
    text_2 = text
    text = text.lower()
    
    #ottengo gli n modelli del testo considerato
    embeddings = []
    for k in range(n_mod):
        model = Word2Vec.load(text + '_' + str(k) + '.model') #adattare nome
        embeddings.append(model) #ottengo una lista contenente gli n modelli per il testo desiderato
        
    #modello per modello, ottengo la lista delle m parole (n_words) più simili al target
    words = []
    for k in range(n_mod):
        for tupla in embeddings[k].wv.most_similar(word)[:n_words]: #ottengo una lista di tuple (parola, similarità)
            words.append(tupla[0]) #appendo solamente la parola, senza considerare il valore di similarità rispetto al target
        
    #creo un dizionario che presenta le parole uniche contenute nella lista "words", e le rispettive frequenze assolute
    c = Counter(words)
    word_frequencies = dict(c)
    
    #creo un dizionario che contiene la lista di parole uniche in "words" e calcola la similarità media di ciascuna di esse
    init = [0] * len(np.unique(words))
    word_similarities = dict(zip(np.unique(words), init))

    for k in range(n_mod):
        for tupla in embeddings[k].wv.most_similar(word)[:n_words]:
            word_similarities[tupla[0]] += tupla[1]
    for word in np.unique(words):
        word_similarities[word] = word_similarities[word] / word_frequencies[word]

    #trasformo il dizionario con le frequenze assolute e quello con le similarità medie in dataframe, e li unisco
    df_1 = pd.DataFrame(list(word_frequencies.items()), columns=['Words','Abs.frequency'])
    df_2 = pd.DataFrame(list(word_similarities.items()), columns=['Words','Med.similarity'])
  
    #calcolo anche la similarità normalizzata, nel caso dovesse servie
    normalized_similarity = (df_2['Med.similarity']-df_2['Med.similarity'].min())/(df_2['Med.similarity'].max() - df_2['Med.similarity'].min()) 
    df = pd.merge(df_1, df_2, on = 'Words')
    df['Med.similarity(norm.)'] = normalized_similarity

    #aggiungo la colonna col nome del testo considerato
    df['Text'] = [text_2] * len(df)
    
    return df

In [92]:
os.chdir(path_autori_cade)

### Virgilio, la guida per Dante, cosa diventa per gli altri autori?

In [54]:
lista_autori_cade = ['CADE_Alessandro Manzoni_cade', 'CADE_Dante Alighieri_cade', 
                     'CADE_Dino Buzzati_cade', 'CADE_Francesco Petrarca_cade', 
                     'CADE_Gabriele D\'Annunzio_cade', 'CADE_Giacomo Leopardi_cade', 
                     'CADE_Giovanni Boccaccio_cade', 'CADE_Giuseppe Parini_cade', 
                     'CADE_Italo Calvino_cade', 'CADE_Italo Svevo_cade', 
                     'CADE_Ludovico Ariosto_cade', 'CADE_Luigi Pirandello_cade', 
                     'CADE_Pier Paolo Pasolini_cade', 'CADE_Primo Levi_cade', 
                     'CADE_Torquato Tasso_cade', 'CADE_Ugo Foscolo_cade', 
                     'CADE_Umberto Eco_cade', 'CADE_Vittorio Alfieri_cade']

In [14]:
dante_virgilio = similar_words_per_topos('virgilio', 5, 10, 'CADE_Dante Alighieri_cade')
print(dante_virgilio[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0   fieramente              5        0.528632               0.251509
1        patre              5        0.494996               0.696622
2     dichiara              3        0.487430               0.609017
3        ficcò              5        0.479052               0.394127
4        dante              2        0.496887               0.641932
5     volgemmo              3        0.475948               0.531411
6       eneida              5        0.490983               0.990677
7     recitare              5        0.486641               0.251454
8     pregasse              1        0.529638               0.401387
9       sicuro              1        0.521403               0.679104
10        enea              1        0.464231               1.000000
11    afflitto              3        0.448835               0.601710
12      stazio              1        0.461227               0.224808
13   ristretti              3     

In [55]:
lista_autori_cade.remove('CADE_Dante Alighieri_cade')

In [9]:
for autore in lista_autori_cade:
        print("### " + autore + " ###")        
        df = compare_word_btw_embeddings("virgilio", 5, 'CADE_Dante Alighieri_cade', autore)
        print(df)

### CADE_Alessandro Manzoni_cade ###
         Words  Med.similarity
8      dannati        0.501943
3     carneade        0.501833
6         cino        0.495997
14  intromesso        0.490050
16  lamentando        0.488214
9     descrive        0.456948
29     vicenza        0.437261
27    studioso        0.431471
17       legne        0.420119
24   sfrontata        0.418347
### CADE_Dino Buzzati_cade ###
              Words  Med.similarity
24         lanciava        0.515920
6           capisse        0.511659
0   affettuosamente        0.510071
36      scalfittura        0.509341
3           ansando        0.505669
18           groppa        0.503361
22          indegno        0.500817
40            sostò        0.500081
14          doverlo        0.463152
5            buffet        0.459956
### CADE_Francesco Petrarca_cade ###
       Words  Med.similarity
15     movea        0.479017
0    achille        0.473731
17       odo        0.472596
11     gridi        0.472538
3      bruti 

### Beatrice dantesca negli altri autori

In [16]:
dante_beatrice = similar_words_per_topos('beatrice', 5, 10, 'CADE_Dante Alighieri_cade')
print(dante_beatrice[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        bernardo              5        0.484054               0.692427
1     manifestano              2        0.488382               0.000000
2       scrivesse              5        0.447429               0.231815
3          rivide              3        0.463385               0.149188
4     contristato              1        0.525315               0.214669
5    ymaginatione              2        0.465498               1.000000
6   gloriosamente              1        0.522955               0.156397
7        gloriosa              3        0.449697               0.166828
8             xxx              5        0.471204               0.436313
9           xxxii              2        0.464881               0.982410
10           xxxi              4        0.430033               0.724686
11       pregasse              2        0.427740               0.057222
12         xxviii              1        0.426246               0

In [18]:
for autore in lista_autori_cade:
        print("### " + autore + " ###")        
        df = compare_word_btw_embeddings("beatrice", 5, 'CADE_Dante Alighieri_cade', autore)
        print(df)

### CADE_Alessandro Manzoni_cade ###
             Words  Med.similarity
17           narra        0.469029
13      magnifiche        0.468624
2         convulsa        0.468134
19           onorò        0.467752
32        sgabelli        0.467015
35        tramonto        0.463232
20        parlarci        0.459956
33         sposava        0.458833
11  innocentemente        0.458358
1         carneade        0.431331
### CADE_Dino Buzzati_cade ###
           Words  Med.similarity
30          sofà        0.481982
22       pupille        0.471113
4         deluso        0.470791
33  strisciavano        0.469642
7      fantoccio        0.467177
5      domandato        0.462096
2      capezzale        0.461571
20       paurosa        0.426226
12     incredulo        0.415663
29      sgomenta        0.395413
### CADE_Francesco Petrarca_cade ###
       Words  Med.similarity
26     quali        0.440021
24     parìs        0.429160
5      bocca        0.423056
22    oreste        0.420336
12

### INFERNO DANTESCO

In [56]:
dante_inferno = similar_words_per_topos('inferno', 5, 10, 'CADE_Dante Alighieri_cade')
print(dante_inferno[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       stagna              5        0.603245               0.339901
1        rosse              5        0.592306               0.388012
2      belzebù              5        0.595014               0.501985
3     descrive              5        0.573032               0.490871
4       porger              1        0.680069               0.079894
5        gaeta              1        0.678990               0.206288
6    scrivesse              1        0.678252               0.000000
7   incomincia              5        0.599127               0.373273
8       cadder              4        0.593116               0.993683
9    terrestre              2        0.594235               0.526065
10     appella              2        0.567332               0.158760
11        pars              1        0.576557               0.393913
12        arli              1        0.575549               0.129790
13     comedia              3     

In [57]:
for autore in lista_autori_cade:
        print("### " + autore + " ###")        
        df = compare_word_btw_embeddings("inferno", 5, 'CADE_Dante Alighieri_cade', autore)
        print(df)

### CADE_Alessandro Manzoni_cade ###
             Words  Med.similarity
2          astorre        0.672062
18  innocentemente        0.669071
17         gerardo        0.668478
35         vistoso        0.667039
30          rosato        0.663904
14          fratte        0.663248
0        andronico        0.662865
7          ciclope        0.659371
4          bagnate        0.659279
16           fusti        0.585804
### CADE_Dino Buzzati_cade ###
           Words  Med.similarity
21         roghi        0.662690
18      prossime        0.661933
4        burroni        0.658656
0   accompagnava        0.655717
17   piedistallo        0.593927
6        culmine        0.590994
19    provenisse        0.587463
8     fatalmente        0.584109
20      riparata        0.575746
9      funeraria        0.572590
### CADE_Francesco Petrarca_cade ###
       Words  Med.similarity
13     gange        0.642876
8      calpe        0.640057
26      reno        0.638797
23       pin        0.638071
33

### PURGATORIO DANTESCO

In [58]:
dante_purgatorio = similar_words_per_topos('purgatorio', 5, 10, 'CADE_Dante Alighieri_cade')
print(dante_purgatorio[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     descrivendo              5        0.731327               0.415452
1        commedia              5        0.714101               1.000000
2           overo              5        0.682811               0.404753
3         comedia              1        0.775251               0.586975
4         cantica              3        0.688706               0.703323
5         comedie              3        0.687122               0.114276
6            pars              5        0.710375               0.443375
7    soprascritta              3        0.701998               0.275992
8          xxxiii              3        0.697437               0.375631
9             est              5        0.692840               0.561805
10     pervennero              4        0.704764               0.523913
11            iii              2        0.668059               0.000000
12  significativo              2        0.640893               0

In [59]:
for autore in lista_autori_cade:
        print("### " + autore + " ###")        
        df = compare_word_btw_embeddings("purgatorio", 5, 'CADE_Dante Alighieri_cade', autore)
        print(df)

### CADE_Alessandro Manzoni_cade ###
          Words  Med.similarity
2      archivio        0.672559
26          seg        0.659306
12          leo        0.650446
31         xxxv        0.630071
19       primus        0.627841
30          xxx        0.610236
10          hic        0.600217
18    peregrini        0.577260
14  necessitate        0.571800
3          area        0.566206
### CADE_Dino Buzzati_cade ###
         Words  Med.similarity
26    scorrere        0.590275
19    notevoli        0.584787
13      giuste        0.578685
25     saletta        0.577493
4   architrave        0.576883
6         club        0.575386
22       piste        0.574841
16      guglia        0.542617
5       buenos        0.517787
1        aereo        0.512653
### CADE_Francesco Petrarca_cade ###
           Words  Med.similarity
22          reno        0.652342
1         andava        0.644664
27        stende        0.637052
0        alzarsi        0.634569
16       palazzi        0.633052
28  

### PARADISO DANTESCO

In [60]:
dante_paradiso = similar_words_per_topos('paradiso', 5, 10, 'CADE_Dante Alighieri_cade')
print(dante_paradiso[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0   contentare              5        0.598089               0.289567
1    terrestre              5        0.543873               1.000000
2      essenza              1        0.579596               0.686240
3         mero              5        0.496438               0.194934
4          xxx              1        0.568415               0.298865
5      piaceri              4        0.489785               0.473109
6      entraro              5        0.494808               0.874459
7       ottavo              1        0.559979               0.412345
8   incomincia              1        0.559178               0.156918
9     descrive              1        0.551870               0.735849
10      escono              1        0.520475               0.309932
11       aiutò              3        0.493438               0.741286
12   figurando              4        0.511524               0.000000
13    divinità              1     

In [61]:
for autore in lista_autori_cade:
        print("### " + autore + " ###")        
        df = compare_word_btw_embeddings("paradiso", 5, 'CADE_Dante Alighieri_cade', autore)
        print(df)

### CADE_Alessandro Manzoni_cade ###
          Words  Med.similarity
7         capit        0.518095
15      gesuita        0.507865
35  successioni        0.506912
24      ottavia        0.505478
34      sognata        0.504948
17          ing        0.504945
13    facendoli        0.502261
27       pavesi        0.501735
10        darli        0.470063
21      machina        0.445134
### CADE_Dino Buzzati_cade ###
           Words  Med.similarity
28   serenissima        0.523365
0   appassionata        0.520057
32         tocco        0.514401
7           crea        0.512624
6       condurla        0.464504
2          bivio        0.460212
5      conducono        0.452932
17       innesto        0.449585
21        pagata        0.444947
30    sopportare        0.439604
### CADE_Francesco Petrarca_cade ###
        Words  Med.similarity
14   naturale        0.532098
22      salir        0.529650
29      sorga        0.517053
13      natio        0.514879
9       corti        0.513082


### Orlando di Ariosto negli altri autori

In [ ]:
lista_autori_cade = ['CADE_Alessandro Manzoni_cade', 'CADE_Dante Alighieri_cade', 
                     'CADE_Dino Buzzati_cade', 'CADE_Francesco Petrarca_cade', 
                     'CADE_Gabriele D\'Annunzio_cade', 'CADE_Giacomo Leopardi_cade', 
                     'CADE_Giovanni Boccaccio_cade', 'CADE_Giuseppe Parini_cade', 
                     'CADE_Italo Calvino_cade', 'CADE_Italo Svevo_cade', 
                     'CADE_Ludovico Ariosto_cade', 'CADE_Luigi Pirandello_cade', 
                     'CADE_Pier Paolo Pasolini_cade', 'CADE_Primo Levi_cade', 
                     'CADE_Torquato Tasso_cade', 'CADE_Ugo Foscolo_cade', 
                     'CADE_Umberto Eco_cade', 'CADE_Vittorio Alfieri_cade']

In [21]:
ariosto_orlando = similar_words_per_topos('orlando', 5, 10, 'CADE_Ludovico Ariosto_cade')
print(ariosto_orlando[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      oliviero              5        0.411753               1.000000
1    fiordiligi              3        0.405563               0.426496
2   abbracciato              1        0.455295               0.654855
3    sacripante              3        0.417499               0.824126
4       olivier              2        0.416205               0.184274
5       inglese              1        0.442221               0.143743
6       capitar              1        0.440031               0.241701
7           lxx              1        0.437937               0.106866
8      bilancia              2        0.405520               0.780258
9      credette              1        0.436223               0.165813
10     capitare              2        0.384497               0.169013
11    stramazza              4        0.394235               0.426998
12      giudicò              1        0.372843               0.187820
13    diligenza     

In [22]:
lista_autori_cade.remove('CADE_Ludovico Ariosto_cade')

In [24]:
for autore in lista_autori_cade:
        print("### " + autore + " ###")        
        df = compare_word_btw_embeddings("orlando", 5, 'CADE_Ludovico Ariosto_cade', autore)
        print(df)

### CADE_Alessandro Manzoni_cade ###
            Words  Med.similarity
32    soccorrerlo        0.422025
5            ardì        0.414249
38          tentò        0.413464
12    confondendo        0.412304
17         invocò        0.410111
0     accoglierlo        0.409447
35     strepitose        0.392216
30      sconfitto        0.382895
9   ciononostante        0.380832
11    comunicarsi        0.378571
### CADE_Dante Alighieri_cade ###
            Words  Med.similarity
23       passarsi        0.412612
37          tentò        0.406693
26         reggia        0.400066
13        fanciul        0.396341
14         ferita        0.394203
15        fermato        0.392868
36         stallo        0.390963
16           gade        0.371684
19  ingiustamente        0.371602
31         scocca        0.360346
### CADE_Dino Buzzati_cade ###
         Words  Med.similarity
35   voltatosi        0.414021
4   fissandolo        0.413905
32     veicolo        0.413408
1      aprirsi        0.41

### Le maschere di Pirandello negli altri autori

In [ ]:
lista_autori_cade = ['CADE_Alessandro Manzoni_cade', 'CADE_Dante Alighieri_cade', 
                     'CADE_Dino Buzzati_cade', 'CADE_Francesco Petrarca_cade', 
                     'CADE_Gabriele D\'Annunzio_cade', 'CADE_Giacomo Leopardi_cade', 
                     'CADE_Giovanni Boccaccio_cade', 'CADE_Giuseppe Parini_cade', 
                     'CADE_Italo Calvino_cade', 'CADE_Italo Svevo_cade', 
                     'CADE_Ludovico Ariosto_cade', 'CADE_Luigi Pirandello_cade', 
                     'CADE_Pier Paolo Pasolini_cade', 'CADE_Primo Levi_cade', 
                     'CADE_Torquato Tasso_cade', 'CADE_Ugo Foscolo_cade', 
                     'CADE_Umberto Eco_cade', 'CADE_Vittorio Alfieri_cade']

In [25]:
pirandello_maschera = similar_words_per_topos('maschera', 5, 10, 'CADE_Luigi Pirandello_cade')
print(pirandello_maschera[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         informe              5        0.515715               0.038900
1   inconsistente              1        0.554545               0.958603
2           bruta              1        0.547342               0.344864
3        mostrerà              1        0.539642               0.242238
4          riveli              4        0.452970               0.437220
5          nudità              1        0.522253               0.314261
6       conferiva              3        0.440555               0.769494
7      distaccare              5        0.435230               0.485495
8     esisterebbe              1        0.514438               0.074974
9        fissarla              3        0.465024               0.186733
10     irrigidita              4        0.442891               1.000000
11           crei              1        0.456624               0.776831
12         scopra              4        0.430069               0

In [27]:
lista_autori_cade.remove('CADE_Luigi Pirandello_cade')

In [28]:
for autore in lista_autori_cade:
        print("### " + autore + " ###")        
        df = compare_word_btw_embeddings("maschera", 5, 'CADE_Luigi Pirandello_cade', autore)
        print(df)

### CADE_Alessandro Manzoni_cade ###
           Words  Med.similarity
3         crassa        0.521892
16  imperterrita        0.519422
4      creazione        0.518935
5        credula        0.503116
6         delusa        0.499908
19      meditata        0.499562
14      immutata        0.498896
21        ottuso        0.495719
20      operante        0.495525
25    scompariva        0.438640
### CADE_Dante Alighieri_cade ###
          Words  Med.similarity
13       figura        0.484128
14     immobile        0.475716
22  signoreggia        0.467974
2        ascosa        0.453767
9        derisa        0.452487
8      converte        0.451036
21        rosso        0.426924
0        ancide        0.425537
25      venereo        0.399929
24     suggello        0.398778
### CADE_Dino Buzzati_cade ###
          Words  Med.similarity
10      macchia        0.544638
22  splendevano        0.518783
24      tettoia        0.513378
25      vastità        0.509161
14    orientale        

### Silvia da Leopardi agli altri autori

In [96]:
lista_autori_cade = ['CADE_Alessandro Manzoni_cade', 'CADE_Dante Alighieri_cade', 
                     'CADE_Dino Buzzati_cade', 'CADE_Francesco Petrarca_cade', 
                     'CADE_Gabriele D\'Annunzio_cade', 'CADE_Giacomo Leopardi_cade', 
                     'CADE_Giovanni Boccaccio_cade', 'CADE_Giuseppe Parini_cade', 
                     'CADE_Italo Calvino_cade', 'CADE_Italo Svevo_cade', 
                     'CADE_Ludovico Ariosto_cade', 'CADE_Luigi Pirandello_cade', 
                     'CADE_Pier Paolo Pasolini_cade', 'CADE_Primo Levi_cade', 
                     'CADE_Torquato Tasso_cade', 'CADE_Ugo Foscolo_cade', 
                     'CADE_Umberto Eco_cade', 'CADE_Vittorio Alfieri_cade']

In [97]:
leopardi_silvia = similar_words_per_topos('silvia', 5, 10, 'CADE_Giacomo Leopardi_cade')
print(leopardi_silvia[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

        Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         xxi              5        0.822247               0.139274
1    rimembri              5        0.848191               0.322646
2     incliti              1        0.821879               0.757558
3   lagrimava              5        0.726322               0.588630
4      rimena              1        0.809475               0.275619
5     divampò              4        0.718138               0.173189
6   polverosi              1        0.806388               0.058139
7       filli              2        0.769207               0.862956
8      traean              2        0.721914               0.365271
9      eletta              1        0.801642               0.053455
10       fura              2        0.709108               0.364659
11     giovin              4        0.689442               0.368743
12    orrende              1        0.726988               0.782274
13   martelli              1        0.726204    

In [94]:
lista_autori_cade.remove('CADE_Giacomo Leopardi_cade')

In [32]:
for autore in lista_autori_cade:
        print("### " + autore + " ###")        
        df = compare_word_btw_embeddings("silvia", 5, 'CADE_Giacomo Leopardi_cade', autore)
        print(df)

### CADE_Alessandro Manzoni_cade ###
           Words  Med.similarity
14    immacolato        0.815707
1            ato        0.785486
30         terse        0.783284
5   concittadino        0.778835
34      valletto        0.778401
13   gorgogliava        0.777678
10        gelato        0.776516
23         sazio        0.771852
36       violato        0.754771
21      rizzarsi        0.724321
### CADE_Dante Alighieri_cade ###
         Words  Med.similarity
21   navicanti        0.687657
33     turbava        0.685545
15     interno        0.683246
5    cerchiamo        0.681201
10      ferito        0.680623
4        canta        0.680531
12      fondar        0.680367
8    disperato        0.679978
13      guasco        0.635292
31  sordamente        0.620376
### CADE_Dino Buzzati_cade ###
         Words  Med.similarity
29      stille        0.755565
0      arrosto        0.753121
27    spensero        0.747991
32   vedendomi        0.746496
26    spelonca        0.744205
25    sp

### Prassede, personaggio dei Promessi Sposi che nell'altra analisi era la parola più simile a "donna" nel romanticismo

In [62]:
lista_autori_cade = ['CADE_Alessandro Manzoni_cade', 'CADE_Dante Alighieri_cade', 
                     'CADE_Dino Buzzati_cade', 'CADE_Francesco Petrarca_cade', 
                     'CADE_Gabriele D\'Annunzio_cade', 'CADE_Giacomo Leopardi_cade', 
                     'CADE_Giovanni Boccaccio_cade', 'CADE_Giuseppe Parini_cade', 
                     'CADE_Italo Calvino_cade', 'CADE_Italo Svevo_cade', 
                     'CADE_Ludovico Ariosto_cade', 'CADE_Luigi Pirandello_cade', 
                     'CADE_Pier Paolo Pasolini_cade', 'CADE_Primo Levi_cade', 
                     'CADE_Torquato Tasso_cade', 'CADE_Ugo Foscolo_cade', 
                     'CADE_Umberto Eco_cade', 'CADE_Vittorio Alfieri_cade']

In [63]:
manzoni_prassede = similar_words_per_topos('prassede', 5, 10, 'CADE_Alessandro Manzoni_cade')
print(manzoni_prassede[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          donna              5        0.652840               0.225034
1       maritate              5        0.569401               0.414496
2       favorita              5        0.552707               0.213030
3    gentildonna              5        0.539687               0.092489
4      governava              5        0.499124               1.000000
5       allogata              5        0.494343               0.510403
6       appagare              5        0.533092               0.446742
7   approfittava              5        0.491888               0.248412
8       occhioni              4        0.488002               0.592028
9        spirare              2        0.475952               0.000000
10     convincer              2        0.467234               0.194033
11          mona              2        0.448319               0.135113


In [64]:
lista_autori_cade.remove('CADE_Alessandro Manzoni_cade')

In [36]:
for autore in lista_autori_cade:
        print("### " + autore + " ###")        
        df = compare_word_btw_embeddings("prassede", 5, 'CADE_Alessandro Manzoni_cade', autore)
        print(df)

### CADE_Dante Alighieri_cade ###
                Words  Med.similarity
18          imaginava        0.455600
28          piacevole        0.453798
3                atta        0.450228
14             famosa        0.439189
17           gloriosa        0.436238
38            vedeste        0.435946
29             poteva        0.435133
2             assegno        0.432732
1   antichissimamente        0.431704
37               sora        0.405508
### CADE_Dino Buzzati_cade ###
             Words  Med.similarity
19         rimaner        0.428253
17     naturalisti        0.421261
20          seduta        0.417905
21          stucco        0.413451
9   frequentemente        0.403501
13         intenta        0.397768
24      verosimile        0.396778
2      confondersi        0.395790
7            donna        0.394070
18       pettegole        0.392228
### CADE_Francesco Petrarca_cade ###
       Words  Med.similarity
12    fisico        0.451935
4       cale        0.450788
27     v

### RENZO E LUCIA

In [41]:
manzoni_renzo = similar_words_per_topos('renzo', 5, 10, 'CADE_Alessandro Manzoni_cade')
print(manzoni_renzo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

              Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0             fermo              5        0.410166               0.000000
1           garzone              5        0.386983               0.276003
2           susurrò              4        0.373103               0.141776
3        trasalendo              2        0.364454               0.140327
4         finiamola              4        0.362072               0.219438
5            spiegò              4        0.362427               0.359506
6            osando              1        0.366721               0.304455
7         colezione              1        0.366218               0.067023
8      afferrandolo              1        0.364420               0.108050
9       andandosene              2        0.355939               1.000000
10          caldaia              2        0.369696               0.238831
11          boccale              1        0.360846               0.633090
12  angosciosamente              2    

In [44]:
for autore in lista_autori_cade:
        print("### " + autore + " ###")        
        df = compare_word_btw_embeddings("renzo", 5, 'CADE_Alessandro Manzoni_cade', autore)
        print(df)

### CADE_Dante Alighieri_cade ###
       Words  Med.similarity
2     atteso        0.333758
23    sonava        0.332947
26      taci        0.330977
10  ficcando        0.329880
1     attese        0.328097
11  giocondo        0.326306
34   vedermi        0.325198
24   specchi        0.324039
9     drizza        0.323902
16     motti        0.323801
### CADE_Dino Buzzati_cade ###
         Words  Med.similarity
12      dritto        0.323793
26      statua        0.320343
10  confidenza        0.319774
18   origliare        0.319627
22     ragazzo        0.319473
15    giocando        0.319302
5        avido        0.318473
28  stupefatto        0.313549
9     capivano        0.312997
25  sostenersi        0.312594
### CADE_Francesco Petrarca_cade ###
        Words  Med.similarity
29   trafisse        0.316664
23     seguir        0.315086
24  sfavillar        0.309412
16      levar        0.308704
20   rattenne        0.305414
22  sdegnando        0.305138
9        ergo        0.30482

In [45]:
manzoni_lucia = similar_words_per_topos('lucia', 5, 10, 'CADE_Alessandro Manzoni_cade')
print(manzoni_lucia[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

              Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0            agnese              5        0.429905               0.333795
1            udendo              1        0.423497               0.275838
2        arrossando              5        0.441069               0.823542
3           dirotto              1        0.417304               0.571010
4     singhiozzando              5        0.422541               1.000000
5         contadina              5        0.399873               0.587633
6   angosciosamente              1        0.413927               0.016724
7         atterrita              5        0.414979               0.348882
8           sperava              3        0.387753               0.298006
9      accostandosi              1        0.395252               0.624388
10        smemorata              3        0.398184               0.105125
11         entrarvi              4        0.384451               0.076626
12       costernata              1    

In [46]:
for autore in lista_autori_cade:
        print("### " + autore + " ###")        
        df = compare_word_btw_embeddings("lucia", 5, 'CADE_Alessandro Manzoni_cade', autore)
        print(df)

### CADE_Dante Alighieri_cade ###
          Words  Med.similarity
14        lucia        0.378662
27       sfatto        0.365583
10      gravoso        0.348678
17      natiche        0.346984
7        eretta        0.345436
29   sospirando        0.344582
4   chiamandomi        0.342737
19    religioso        0.342293
2        bagnar        0.341780
0   altrettanti        0.341659
### CADE_Dino Buzzati_cade ###
         Words  Med.similarity
4   compiaccio        0.342576
7       covare        0.341431
25     rotelle        0.337225
9    dominarsi        0.335505
13       luisa        0.335213
28   solletico        0.334811
23     ragazza        0.334086
10   esultante        0.333164
19     partire        0.333117
6   confidenza        0.332398
### CADE_Francesco Petrarca_cade ###
         Words  Med.similarity
21      rimasi        0.349219
30     vivendo        0.343559
16  lungamente        0.339792
13       dicea        0.338294
14     fattore        0.338125
1   altramente     

### I BRAVI DI MANZONI

In [47]:
manzoni_bravi = similar_words_per_topos('bravi', 5, 10, 'CADE_Alessandro Manzoni_cade')
print(manzoni_bravi[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

              Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         vagabondi              5        0.487691               0.426755
1   accompagnamento              3        0.421896               0.146997
2            forche              1        0.457767               0.189829
3          scendono              5        0.421966               0.508431
4      scompigliata              1        0.440509               0.739283
5         servitori              1        0.436894               0.137326
6        trasmutato              1        0.436462               0.156425
7        travestiti              5        0.422109               0.000000
8             ronda              2        0.415100               0.195447
9          camminar              1        0.431270               0.090537
10          zingari              4        0.391185               0.367546
11      sopraddetti              1        0.398761               0.427367
12            bando              1    

In [48]:
for autore in lista_autori_cade:
        print("### " + autore + " ###")        
        df = compare_word_btw_embeddings("bravi", 5, 'CADE_Alessandro Manzoni_cade', autore)
        print(df)

### CADE_Dante Alighieri_cade ###
         Words  Med.similarity
6      cotanti        0.390381
34    sentieri        0.377587
2     ascoltar        0.376677
1       andato        0.368452
32   riguardar        0.368100
11    fendendo        0.367673
21    leonessa        0.365578
28      pareti        0.363538
25        nudi        0.350672
7   dannazione        0.339110
### CADE_Dino Buzzati_cade ###
          Words  Med.similarity
8      bifolchi        0.380241
19        fermi        0.372929
34  valloncello        0.363631
11     cisterna        0.362947
20  formidabili        0.361768
14       cresta        0.359099
10        carri        0.358006
29       scorta        0.346199
3    arrivarono        0.340437
12    cocchiere        0.333421
### CADE_Francesco Petrarca_cade ###
         Words  Med.similarity
15       padri        0.371046
4     chiudere        0.358126
2        amici        0.356788
20  seguitarla        0.355340
22     sermone        0.355329
23       serpi     

### DON RODRIGO

In [65]:
manzoni_rodrigo = similar_words_per_topos('rodrigo', 5, 10, 'CADE_Alessandro Manzoni_cade')
print(manzoni_rodrigo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0              don              5        0.800897               0.340865
1         abbondio              5        0.549735               0.029464
2          vincono              5        0.466460               0.000000
3       palazzotto              1        0.450768               0.036016
4         ferrante              2        0.442947               1.000000
5          gonzalo              1        0.441506               0.060616
6           riesci              4        0.437281               0.025063
7             vali              5        0.444460               0.056834
8     sogghignando              2        0.423504               0.054637
9     sghignazzava              5        0.434400               0.081140
10          scotta              2        0.439873               0.060173
11         bramava              1        0.431076               0.045747
12         parlavi              3        0.442778  

In [67]:
for autore in lista_autori_cade:
        print("### " + autore + " ###")        
        df = compare_word_btw_embeddings("rodrigo", 5, 'CADE_Alessandro Manzoni_cade', autore)
        print(df)

### CADE_Dante Alighieri_cade ###
           Words  Med.similarity
38          xlvi        0.342310
21          pira        0.342277
32  significasse        0.340301
16         monda        0.340034
11   frequentato        0.339351
0         aguzzo        0.339207
9        fermati        0.338350
22    ribalderia        0.335505
15         matto        0.333885
10         forca        0.333359
### CADE_Dino Buzzati_cade ###
          Words  Med.similarity
7           don        0.485644
3       anselmo        0.377532
25      valerio        0.369314
16     raimondo        0.358889
24    valentino        0.350574
20   sostenersi        0.346670
19   soavemente        0.345708
15     prevosto        0.344808
8   fluttuavano        0.343999
0   accomodarsi        0.342347
### CADE_Francesco Petrarca_cade ###
        Words  Med.similarity
27   traditor        0.340174
14     lecito        0.335814
13      lasce        0.325809
16       muta        0.323637
2     angusta        0.323588
9  

### DON ABBONDIO

In [68]:
manzoni_abbondio = similar_words_per_topos('abbondio', 5, 10, 'CADE_Alessandro Manzoni_cade')
print(manzoni_abbondio[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0             don              5        0.853032               0.099625
1         rodrigo              5        0.549735               0.023580
2        ferrante              5        0.498920               1.000000
3       valeriano              3        0.441473               0.227930
4   svincolandosi              1        0.444979               0.091849
5      gongolante              5        0.447543               0.092495
6         rilesse              1        0.438373               0.115914
7       frangenti              1        0.436506               0.043971
8      spaventato              1        0.435043               0.000000
9         gonzalo              2        0.414546               0.143453
10          sella              4        0.447465               0.081423
11            din              1        0.405194               0.086846
12         scotta              1        0.395130               0

In [69]:
for autore in lista_autori_cade:
        print("### " + autore + " ###")        
        df = compare_word_btw_embeddings("abbondio", 5, 'CADE_Alessandro Manzoni_cade', autore)
        print(df)

### CADE_Dante Alighieri_cade ###
           Words  Med.similarity
2          anelo        0.356667
5          balìa        0.354037
31  significasse        0.348673
19         latin        0.347235
1         aguzzo        0.346286
17       imagina        0.342642
29         scale        0.337414
33         tremò        0.337063
9          cipri        0.336720
22         marra        0.335993
### CADE_Dino Buzzati_cade ###
        Words  Med.similarity
3         don        0.499732
22  valentino        0.392023
1     anselmo        0.385129
20   umiliato        0.380015
8      irritò        0.371067
16   raimondo        0.370666
15   prevosto        0.366854
23    valerio        0.366635
2    cilindro        0.366311
5   incredulo        0.362610
### CADE_Francesco Petrarca_cade ###
        Words  Med.similarity
19  manifesta        0.355336
18     lecito        0.350760
28      volgo        0.347688
8     disegnò        0.342355
26      uscia        0.336130
24        toi        0.33

### Il piacere per D'Annunzio negli altri autori

In [ ]:
lista_autori_cade = ['CADE_Alessandro Manzoni_cade', 'CADE_Dante Alighieri_cade', 
                     'CADE_Dino Buzzati_cade', 'CADE_Francesco Petrarca_cade', 
                     'CADE_Gabriele D\'Annunzio_cade', 'CADE_Giacomo Leopardi_cade', 
                     'CADE_Giovanni Boccaccio_cade', 'CADE_Giuseppe Parini_cade', 
                     'CADE_Italo Calvino_cade', 'CADE_Italo Svevo_cade', 
                     'CADE_Ludovico Ariosto_cade', 'CADE_Luigi Pirandello_cade', 
                     'CADE_Pier Paolo Pasolini_cade', 'CADE_Primo Levi_cade', 
                     'CADE_Torquato Tasso_cade', 'CADE_Ugo Foscolo_cade', 
                     'CADE_Umberto Eco_cade', 'CADE_Vittorio Alfieri_cade']

In [37]:
dannunzio_piacere = similar_words_per_topos('piacere', 5, 10, 'CADE_Gabriele D\'Annunzio_cade')
print(dannunzio_piacere[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        palesato              5        0.375627               0.411207
1         sedotta              2        0.396439               0.896196
2         diletto              4        0.368511               0.930187
3      parlandole              1        0.434622               0.106353
4        generava              1        0.433696               0.391026
5   compiacimento              1        0.426840               0.000000
6        sfrenato              5        0.384884               0.194897
7     vendicativo              5        0.365409               0.406943
8      ricordarle              5        0.368831               0.149019
9        attirare              1        0.423051               0.173756
10     ascoltarlo              1        0.368986               0.991693
11      scriverle              3        0.349915               0.243367
12       sollievo              1        0.341606               0

In [39]:
lista_autori_cade.remove('CADE_Gabriele D\'Annunzio_cade')

In [40]:
for autore in lista_autori_cade:
        print("### " + autore + " ###")        
        df = compare_word_btw_embeddings("piacere", 5, 'CADE_Gabriele D\'Annunzio_cade', autore)
        print(df)

### CADE_Alessandro Manzoni_cade ###
            Words  Med.similarity
18        piacere        0.623585
23       provarne        0.420092
0      accrescere        0.419824
20       premiare        0.419472
7   compiacendosi        0.413647
25       proverei        0.412713
6         colmare        0.409655
37      vivissimo        0.408187
3       apportato        0.407433
26          provo        0.371343
### CADE_Dante Alighieri_cade ###
         Words  Med.similarity
18     piacere        0.458011
17   perdonare        0.396136
7    consentir        0.395991
1        amato        0.391722
0         amar        0.389887
31   trascende        0.386779
12  guiderdone        0.384188
10     gioioso        0.356319
4       cambio        0.344005
13    innamora        0.340109
### CADE_Dino Buzzati_cade ###
         Words  Med.similarity
0   accusatore        0.415968
15    piangere        0.405748
4      balocco        0.402615
21   riuscirvi        0.399570
28        zelo        0.3995

### TENTATIVO MOLTO AMBIZIOSO: UOMO DA PRIMO LEVI AGLI ALTRI AUTORI

In [50]:
lista_autori_cade = ['CADE_Alessandro Manzoni_cade', 'CADE_Dante Alighieri_cade', 
                     'CADE_Dino Buzzati_cade', 'CADE_Francesco Petrarca_cade', 
                     'CADE_Gabriele D\'Annunzio_cade', 'CADE_Giacomo Leopardi_cade', 
                     'CADE_Giovanni Boccaccio_cade', 'CADE_Giuseppe Parini_cade', 
                     'CADE_Italo Calvino_cade', 'CADE_Italo Svevo_cade', 
                     'CADE_Ludovico Ariosto_cade', 'CADE_Luigi Pirandello_cade', 
                     'CADE_Pier Paolo Pasolini_cade', 'CADE_Primo Levi_cade', 
                     'CADE_Torquato Tasso_cade', 'CADE_Ugo Foscolo_cade', 
                     'CADE_Umberto Eco_cade', 'CADE_Vittorio Alfieri_cade']

In [51]:
levi_uomo = similar_words_per_topos('uomo', 5, 10, 'CADE_Primo Levi_cade')
print(levi_uomo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       spiccavano              5        0.410713               0.017833
1           autore              5        0.430617               0.000000
2           storie              3        0.413046               0.588218
3         rivelato              5        0.405345               0.198873
4   indispensabile              1        0.482102               0.944018
5        scrittore              1        0.476505               0.105699
6           chiede              1        0.475103               0.118864
7          toscani              1        0.474722               0.318762
8       letterario              2        0.447147               0.209574
9        impotenza              4        0.408259               0.175977
10      espressivi              3        0.383274               0.409400
11      sanguinose              1        0.396979               1.000000
12        epigrafe              1        0.396926  

In [52]:
lista_autori_cade.remove('CADE_Primo Levi_cade')

In [53]:
for autore in lista_autori_cade:
        print("### " + autore + " ###")        
        df = compare_word_btw_embeddings("uomo", 5, 'CADE_Primo Levi_cade', autore)
        print(df)

### CADE_Alessandro Manzoni_cade ###
          Words  Med.similarity
31         uomo        0.499845
1    acutamente        0.446105
15   insistente        0.439751
7    coraggioso        0.439366
14      insigne        0.437080
16      licurgo        0.429179
2        ammira        0.425945
17    peccatori        0.425282
11   giudizioso        0.394055
8   discendente        0.374931
### CADE_Dante Alighieri_cade ###
              Words  Med.similarity
8           inganni        0.433286
14          pargolo        0.430200
16          piccolo        0.429813
17        pittagora        0.420849
4              bono        0.419833
30             uomo        0.406298
22           satiro        0.394736
20           pudico        0.375057
6   eccellentissimo        0.369085
1             asino        0.359281
### CADE_Dino Buzzati_cade ###
            Words  Med.similarity
18  giustificarsi        0.451923
4          astuto        0.450946
3      armistizio        0.446320
5        avvoc

### ZENO DA SVEVO AGLI ALTRI AUTORI

In [70]:
lista_autori_cade = ['CADE_Alessandro Manzoni_cade', 'CADE_Dante Alighieri_cade', 
                     'CADE_Dino Buzzati_cade', 'CADE_Francesco Petrarca_cade', 
                     'CADE_Gabriele D\'Annunzio_cade', 'CADE_Giacomo Leopardi_cade', 
                     'CADE_Giovanni Boccaccio_cade', 'CADE_Giuseppe Parini_cade', 
                     'CADE_Italo Calvino_cade', 'CADE_Italo Svevo_cade', 
                     'CADE_Ludovico Ariosto_cade', 'CADE_Luigi Pirandello_cade', 
                     'CADE_Pier Paolo Pasolini_cade', 'CADE_Primo Levi_cade', 
                     'CADE_Torquato Tasso_cade', 'CADE_Ugo Foscolo_cade', 
                     'CADE_Umberto Eco_cade', 'CADE_Vittorio Alfieri_cade']

In [71]:
svevo_zeno = similar_words_per_topos('zeno', 5, 10, 'CADE_Italo Svevo_cade')
print(svevo_zeno[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          dovete              5        0.586418               0.314717
1       nominarmi              5        0.550197               0.561060
2       domandalo              2        0.581379               0.317109
3        chiamerò              1        0.615801               0.000000
4       pacatezza              1        0.615372               1.000000
5      scotendosi              1        0.614306               0.114742
6       pensateci              5        0.566155               0.451166
7        rammento              1        0.613066               0.061308
8        lascialo              1        0.611171               0.500898
9      scherziamo              1        0.609973               0.727532
10  dispiacerebbe              4        0.552747               0.767418
11             im              4        0.585933               0.763576
12          dario              3        0.546464               0

In [72]:
lista_autori_cade.remove('CADE_Italo Svevo_cade')

In [73]:
for autore in lista_autori_cade:
        print("### " + autore + " ###")        
        df = compare_word_btw_embeddings("zeno", 5, 'CADE_Italo Svevo_cade', autore)
        print(df)

### CADE_Alessandro Manzoni_cade ###
          Words  Med.similarity
20     mettessi        0.592803
31     ringrazi        0.583873
1       aiutate        0.583317
42       trechi        0.581335
35   secretario        0.579753
7     chiamarmi        0.577940
8       cognato        0.576931
39  tenerissimo        0.576874
10       cugine        0.576391
11      ditegli        0.543612
### CADE_Dante Alighieri_cade ###
       Words  Med.similarity
11    famoso        0.551466
20     novel        0.550960
2     arridi        0.541150
25    priego        0.540928
7      dammi        0.538026
32   vedesti        0.535603
21   parlami        0.519840
10        eo        0.513434
29  servente        0.505001
5     crudel        0.502317
### CADE_Dino Buzzati_cade ###
           Words  Med.similarity
7   composizione        0.635710
31          totò        0.624540
0         adolfo        0.622654
5      brutalità        0.621140
10        ditemi        0.619232
17        gliele        0.614

### FOSCOLO: PATRIA E SEPOLCRI (MORTE)

In [74]:
lista_autori_cade = ['CADE_Alessandro Manzoni_cade', 'CADE_Dante Alighieri_cade', 
                     'CADE_Dino Buzzati_cade', 'CADE_Francesco Petrarca_cade', 
                     'CADE_Gabriele D\'Annunzio_cade', 'CADE_Giacomo Leopardi_cade', 
                     'CADE_Giovanni Boccaccio_cade', 'CADE_Giuseppe Parini_cade', 
                     'CADE_Italo Calvino_cade', 'CADE_Italo Svevo_cade', 
                     'CADE_Ludovico Ariosto_cade', 'CADE_Luigi Pirandello_cade', 
                     'CADE_Pier Paolo Pasolini_cade', 'CADE_Primo Levi_cade', 
                     'CADE_Torquato Tasso_cade', 'CADE_Ugo Foscolo_cade', 
                     'CADE_Umberto Eco_cade', 'CADE_Vittorio Alfieri_cade']

In [77]:
lista_autori_cade.remove('CADE_Ugo Foscolo_cade')

In [78]:
foscolo_patria = similar_words_per_topos('patria', 5, 10, 'CADE_Ugo Foscolo_cade')
print(foscolo_patria[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

              Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0           credimi              2        0.501226               0.131669
1            piango              5        0.489013               0.894199
2      ringraziarti              1        0.542368               0.061847
3            chiudi              1        0.527899               0.699170
4      secretamente              5        0.463214               0.870402
5         difficili              1        0.526675               0.850852
6             degno              1        0.524644               0.885250
7          desiderj              1        0.521970               0.172423
8            temute              1        0.520653               0.552472
9               età              2        0.481163               0.141679
10       sacrificio              4        0.442868               0.000000
11       usurpatore              3        0.452862               0.110941
12            animi              4    

In [79]:
for autore in lista_autori_cade:
        print("### " + autore + " ###")        
        df = compare_word_btw_embeddings("patria", 5, 'CADE_Ugo Foscolo_cade', autore)
        print(df)

### CADE_Alessandro Manzoni_cade ###
           Words  Med.similarity
2     bastassero        0.517721
23     investite        0.516125
18      figlioli        0.512020
29       prevale        0.510830
6      chiamarvi        0.510207
25      offrirti        0.508237
30  procacciarmi        0.506238
32     riposarmi        0.506011
13    conoscersi        0.502909
1        assolto        0.464642
### CADE_Dante Alighieri_cade ###
         Words  Med.similarity
10  conservate        0.516241
12  correggere        0.515600
38  tormentano        0.505254
33    soffrire        0.504839
26   paradosso        0.497646
39    torquato        0.497291
34    sollazzo        0.496234
9     conformi        0.495766
17   dilettare        0.494908
1       accusi        0.493406
### CADE_Dino Buzzati_cade ###
            Words  Med.similarity
24      riceverla        0.544482
29        simbolo        0.539437
13      macchiata        0.537809
22     presenterà        0.534796
30          soldo       

In [82]:
foscolo_sepolcri = similar_words_per_topos('sepolcro', 5, 10, 'CADE_Ugo Foscolo_cade')
print(foscolo_sepolcri[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       essermi              5        0.533927               0.248919
1      ritraggo              2        0.557633               0.310125
2     vantaggio              5        0.552324               0.414654
3         tocco              1        0.623875               0.008318
4          beva              5        0.508112               0.311104
5    vecchiezza              1        0.620281               0.010170
6       manderà              5        0.503400               0.146094
7        perder              1        0.610475               0.545185
8       degnato              5        0.487634               0.923595
9          godo              1        0.608765               0.363928
10    ascoltami              4        0.487440               0.003982
11       romita              2        0.485741               0.390828
12    implorato              1        0.498081               0.932240
13  abbandonata     

In [85]:
for autore in lista_autori_cade:
        print("### " + autore + " ###")        
        df = compare_word_btw_embeddings("sepolcro", 5, 'CADE_Ugo Foscolo_cade', autore)
        print(df)

### CADE_Alessandro Manzoni_cade ###
              Words  Med.similarity
18         mitigare        0.629851
10       continenza        0.614668
12         dobbiate        0.612462
30              toc        0.611713
17  indistruttibili        0.611516
28         settanta        0.610776
29            spiar        0.610526
27       seguiranno        0.567957
3          avendoli        0.524874
8           carlino        0.515263
### CADE_Dante Alighieri_cade ###
        Words  Med.similarity
24       mori        0.613428
7   avanzando        0.612381
6      assiso        0.612304
16      fogna        0.610570
15       diva        0.606164
25      oblio        0.603863
23   mantenga        0.603174
19   immobili        0.602601
3          am        0.601109
30  rivestita        0.514952
### CADE_Dino Buzzati_cade ###
           Words  Med.similarity
9          elica        0.614211
2     affrettare        0.610491
18  perennemente        0.606547
33     umilmente        0.605840
12   ge

In [86]:
foscolo_morte = similar_words_per_topos('morte', 5, 10, 'CADE_Ugo Foscolo_cade')
print(foscolo_morte[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

        Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       cenni              5        0.444173               0.869322
1     pianger              1        0.510292               0.170957
2          ah              1        0.503799               0.597390
3      ucciso              5        0.415630               0.954501
4        daga              1        0.502046               0.076414
5         man              1        0.498057               0.536688
6     svelato              1        0.497048               0.942214
7      abbiam              1        0.491643               0.213683
8      acerbo              2        0.452836               0.064512
9       pegno              1        0.487497               0.000000
10     perder              1        0.429171               0.329207
11      quest              2        0.407880               0.914267
12      resti              4        0.407217               0.130626
13    accetta              2        0.391980    

In [87]:
for autore in lista_autori_cade:
        print("### " + autore + " ###")        
        df = compare_word_btw_embeddings("morte", 5, 'CADE_Ugo Foscolo_cade', autore)
        print(df)

### CADE_Alessandro Manzoni_cade ###
         Words  Med.similarity
15       facil        0.518928
13   converria        0.517001
27     repulsa        0.509708
31       tenni        0.507072
2   annunciata        0.465186
22      parean        0.454661
14     disleal        0.451228
5     cadavero        0.450194
30  salutevole        0.449317
21       morte        0.442716
### CADE_Dante Alighieri_cade ###
        Words  Med.similarity
2       argia        0.478352
12       esti        0.474784
26  ricevetti        0.472568
20    passion        0.472103
25     reggia        0.471483
10   discordi        0.471222
19      muoia        0.470773
18      morte        0.430465
11      ecuba        0.418766
5     bestial        0.415636
### CADE_Dino Buzzati_cade ###
         Words  Med.similarity
0    accorgere        0.511062
30      passar        0.497173
27     opporsi        0.495716
18      feriti        0.494909
2   affaticato        0.494351
13    divorare        0.493622
38    spez

### Laura di Petrarca

In [98]:
lista_autori_cade = ['CADE_Alessandro Manzoni_cade', 'CADE_Dante Alighieri_cade', 
                     'CADE_Dino Buzzati_cade', 'CADE_Francesco Petrarca_cade', 
                     'CADE_Gabriele D\'Annunzio_cade', 'CADE_Giacomo Leopardi_cade', 
                     'CADE_Giovanni Boccaccio_cade', 'CADE_Giuseppe Parini_cade', 
                     'CADE_Italo Calvino_cade', 'CADE_Italo Svevo_cade', 
                     'CADE_Ludovico Ariosto_cade', 'CADE_Luigi Pirandello_cade', 
                     'CADE_Pier Paolo Pasolini_cade', 'CADE_Primo Levi_cade', 
                     'CADE_Torquato Tasso_cade', 'CADE_Ugo Foscolo_cade', 
                     'CADE_Umberto Eco_cade', 'CADE_Vittorio Alfieri_cade']

In [99]:
lista_autori_cade.remove('CADE_Francesco Petrarca_cade')

In [100]:
petrarca_laura = similar_words_per_topos('laura', 5, 10, 'CADE_Ugo Foscolo_cade')
print(petrarca_laura[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         onor              5        0.820572               0.235164
1      angusto              1        0.852837               0.214759
2     infranto              5        0.750442               1.000000
3       invase              1        0.837391               0.847988
4    peregrini              5        0.754155               0.846053
5      ricetta              1        0.829095               0.430696
6     raccolse              1        0.828960               0.224238
7      avvolti              1        0.826687               0.404766
8    conoscono              1        0.826354               0.910214
9       spande              1        0.826004               0.466946
10       tocca              4        0.718574               0.303216
11       italo              1        0.761138               0.000000
12  accoglieva              2        0.717756               0.812440
13      rossor              1     

In [101]:
for autore in lista_autori_cade:
        print("### " + autore + " ###")        
        df = compare_word_btw_embeddings("laura", 5, 'CADE_Francesco Petrarca_cade', autore)
        print(df)

### CADE_Alessandro Manzoni_cade ###
           Words  Med.similarity
18       moleste        0.778438
24       rendevo        0.778283
22        prezzi        0.773983
13      formarla        0.772404
32         speri        0.770638
1      biasimare        0.769768
6     confidarmi        0.769439
26  risparmiarti        0.769138
28         sedie        0.768588
21    peggiorato        0.699908
### CADE_Dante Alighieri_cade ###
          Words  Med.similarity
10  disiderando        0.812940
33    spergiuri        0.811817
5    componendo        0.808458
7        coprir        0.805650
1      assalito        0.804530
32   soddisfare        0.804190
9         diria        0.802045
34       tacerò        0.800290
21   nascondere        0.760211
18         lodi        0.759490
### CADE_Dino Buzzati_cade ###
             Words  Med.similarity
20        leggerla        0.800233
11       disordini        0.792021
18       lamentano        0.786851
28        ostilità        0.781133
34  ring